In [1]:
import socketserver

In [2]:
from http.server import SimpleHTTPRequestHandler as SimpleHandler

In [ ]:
port = 9000
socketserver.TCPServer.allow_reuse_address = True
http = socketserver.TCPServer(('localhost',port), SimpleHandler)
print (f'serving on port{port}')
http.serve_forever()

serving on port9000


In [1]:
import pandas as pd
df = pd.read_csv('../data/dSST.csv')
df.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,J-D,D-N,DJF,MAM,JJA,SON
0,1881,-0.18,-0.13,0.04,0.06,0.07,-0.17,0.02,-0.02,-0.14,-0.21,-0.17,-0.06,-0.07,-0.08,-0.16,0.06,-0.06,-0.17
1,1882,0.17,0.15,0.05,-0.16,-0.13,-0.22,-0.15,-0.06,-0.13,-0.23,-0.15,-0.36,-0.10,-0.08,0.09,-0.08,-0.15,-0.17
2,1883,-0.28,-0.36,-0.12,-0.18,-0.17,-0.06,-0.06,-0.13,-0.21,-0.11,-0.23,-0.11,-0.17,-0.19,-0.33,-0.15,-0.09,-0.19
3,1884,-0.12,-0.07,-0.36,-0.39,-0.33,-0.34,-0.32,-0.27,-0.26,-0.24,-0.33,-0.30,-0.28,-0.26,-0.10,-0.36,-0.31,-0.28
4,1885,-0.58,-0.33,-0.26,-0.41,-0.45,-0.43,-0.33,-0.31,-0.28,-0.23,-0.23,-0.10,-0.33,-0.34,-0.40,-0.37,-0.36,-0.25
